In [1]:
#!pip install facebook-sdk

In [14]:
import facebook
import json
from datetime import datetime
import time
from collections import defaultdict
import pandas as pd

In [16]:
class FacebookCollector:
    def __init__(self, access_token):
        try:
            self.access_token = access_token
            self.graph = facebook.GraphAPI(access_token)
        except Exception as e:
            print(f"Initialization Error: {e}")

    def check_token_validity(self):
        try:
            me = self.graph.get_object("me", fields="id,name")
            print("Token is valid")
            return True
        except Exception as e:
            print(f"Token is invalid: {e}")
            return False
    
    def collect_data(self, limit=5):
        try:
            fields = (
                'id'
                ',message'
                ',comments.limit(100).summary(true)'
                '{created_time,from{id,name},message,reactions}'
                ',reactions.limit(100).summary(true)'
                ',{id,type,name}'
                ',share'
                ',type'
            )
            posts = self.graph.get_object(
                "me/feed"
            )
            for post in posts.get('data', []):
                print('-----------------')
                print("id")
                print(post.get('id'))
                print("message")
                print(post.get('message'))
                print('created_time')
                print(post.get('created_time'))
                print('-----------------')
            return posts
        except Exception as e:
            pass
        
    def json_to_excel(self, my_json, excel_file=None):
        try:
            posts = []
            for post in my_json['data']:
                post_data = {
                    'id': post.get('id', ''),
                    'message': post.get('message', ''),
                    'created_time': post.get('created_time', '')
                }
                posts.append(post_data)

            df = pd.DataFrame(posts)
            df['created_time'] = pd.to_datetime(df['created_time']).dt.strftime('%Y-%m-%d %H:%M:%S')

            if not excel_file:
                timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
                excel_file = f'facebook_posts_{timestamp}.csv'
            df.to_csv(excel_file, index=False)
            print(f"Data saved to {excel_file}")
            return excel_file
        except Exception as e:
            print(f"Error exporting to Excel: {e}")
            return None



In [17]:
def main():
    ACCESS_TOKEN= 'EAAM4HEZCvwSUBO6pAP9MTDdXgqZBKgO9laJqeTnSBZAZBjxLmD8BJreyWauQ7xzzT3dqrmRCQilmSEmca4kUzxgWRi9nPtwKdFX02DPWoWNrWRjdx4U0mDb4Bc9ouO2KAZCxDRi51sWFcKaBbW13nfZBSnMzSkVyNqhuEty4SJ9rc1QbZBlV9DHH1TDN6sFFB4RroyMCz7UGNPwaHsLs6MZD'
    collector = FacebookCollector(ACCESS_TOKEN)
    print(collector)    
    if(collector.check_token_validity()):
        data = collector.collect_data(limit = 2)
        collector.json_to_excel(data)
        
        
        
if __name__ == "__main__":
    main()

Token is invalid: Error validating access token: Session has expired on Monday, 25-Nov-24 19:00:00 PST. The current time is Tuesday, 26-Nov-24 04:57:02 PST.
